Estudiante 1:

Estudiante 2:



<p>
<img src=./img/Logo_Uniandes.svg width="20%">
</p>

# Departamento de Ingeniería Mecánica
## IMEC 2001 - Herramientas Computacionales
### Taller 1 - Interpolación y Raices

----
### **<span style="color:#A1CCD1;">Condiciones de la entrega </span>**
----
**1. Fechas de entrega:**
- Nota sobre 5.00 ->  <span style="color:red;">5 de Noviembre a las 23:59, 2023</span>
- Nota sobre 4.00 ->  <span style="color:red;">6 de Noviembre a las 23:59, 2023</span>

**2. Colocar el nombre de los integrantes en la casilla correspondiente<br>**
**3. La entrega del taller es por Bloque Neon y se envía el archivo en formato de Jupyter Notebook (extensión `.ipynb`)<br>**
**4. El nombre del archivo debe ser:** 
- Si trabajó en parejas:`T#_ApellidosIntegrante1_ApellidosIntegrante2` (Ej: `T#_Salamanca_VargasTorres`)
- Si trabajó individual: `T#_Apellido` (Ej: `T#_VargasTorres`)

**5. Comente el código con el proceso que realizó para llegar a la solución.<br>**
**6. <span style="color:red;"> Si se detecta copia con otro de los talleres enviados, automáticamente la nota para los grupos involucrados es 0. </span>**


----
### **<span style="color:#A1CCD1;">Condiciones para el segundo envío </span>**
----
<div class='alert alert-block alert-info'>   

<i class='fa fa-info-circle' aria-hidden='true'></i>
Para poder realizar un segundo envío con correcciones, la nota del **primer envío** debe estar mínimo en **Aprendiz**.
</div>

----
### **Tabla de contenido**
----
### Sección 1: Interpolación [→](#section1)
- 1.1. Punto 1: Interpolación puntos aleatorios
- 1.2. <font color="#276C9B"> Punto 2: Interpolación ciencia de materiales </font> [→](#exercise1)

### Sección 2: Raíces – Solución de Ecuaciones No Lineales [→](#section2)
- 2.1. Raices identificando los puntos 
- 2.2. Esfuerzo de fluencia


In [1]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# No edite este código
def formato_grafica(ax, fig, titulo= "", x_label= "", y_label= "", leyenda=False, xlim=[None, None], ylim=[None, None]):
    
    ax.set_title(titulo)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    
    ax.tick_params(direction='out', length=5, width=0.75, grid_alpha=0.3)
    
    # Utilizarlo si se necesita alguna rotación en los labels de los ejes
#     ax.set_xticklabels(ax.get_xticks(), rotation = 0)
#     ax.set_yticklabels(ax.get_yticks(), rotation = 0)
    ax.set_xlim(xlim[0], xlim[1])
    ax.set_ylim(ylim[0], ylim[1])

    # Grid de la gráfica
    ax.grid(True)
    ax.grid(visible=True, which='major', color='grey', linestyle='-')
    ax.minorticks_on()
    ax.grid(visible=True, which='minor', color='lightgrey', linestyle='-', alpha=0.2)

    if leyenda == True:
        ax.legend(loc='best', fontsize= 7)

    fig.tight_layout()



## **Punto 1 - Sección Interpolación**
----
<div class='alert alert-block alert-info'>   

<i class='fa fa-info-circle' aria-hidden='true'></i>
A continuación tiene 3 conjuntos de datos para los cuales debe:
1. Utiliza la función de numpy np.arange para generar los puntos x de cada conjunto, estos van de 0 a la longitud de cada uno. 
2. Grafique cada conjunto de datos utilizando [``plt.scatter()``](https://realpython.com/visualizing-python-plt-scatter/)
3. Como puede observar, cada gráfica tiene un comportamiento diferente, teniendo que hay un método de interpolación que da una mejor aproximación. Seleccione **3** métodos de interpolación **uno** para cada conjunto de datos (deben ser distintos) y realice la interpolación. 
4. Grafique en un subplot los puntos que utilizó para la interpolación y los que obtuvo para cada conjunto. 

**Bono:** Para el punto 3 utilizar las funciones de las interpolaciones en una función, intente generalizarla (que funcione para cualqueir caso). Siga el siguiente formato:
``` Python
def fun_1(param_1: tipo, param_2: tipo, ....)-> tipo que se retorna:
    """
    Descripción de la función

    Parameters
    ----------
    param_1 : tipo
        Descripción del parámetro
    param_2 : tipo
        Descripción del parámetro

    Returns
    -------
    variable : tipo
        Descripción de la variable 
    """
    Variables y operaciones con el método de interpolación seleccionado
```
</div>



In [3]:
# Conjuntos de datos
datos_1 = np.array([  0. ,   62.5,  250. ,  562.5, 1000.])
datos_2 = np.array([0.00000000e+00, 9.82812500e+02, 3.12750000e+04, 2.37360938e+05,
                    1.00010000e+06, 3.05191406e+06, 7.59397500e+06, 1.64133922e+07,
                    3.20004000e+07])
datos_3 = np.array([0.        ,   6.12244898,  12.24489796,  18.36734694,
                    24.48979592,  30.6122449 ,  36.73469388,  42.85714286,
                    48.97959184,  55.10204082,  61.2244898 ,  67.34693878,
                    73.46938776,  79.59183673,  85.71428571,  91.83673469,
                    97.95918367, 104.08163265, 110.20408163])

----
## Punto 2
----

Algunas de las cargas que se pueden encontrar en la ingeniería son de tensión, compresión, torsión y cortante, para el taller se trabajará con el primero. Una de las pruebas que se le realizan a los materiales es la prueba de tensión en donde se deforma la probeta aplicando una carga axial hasta que se llega a la fractura como se muestra en la siguiente imagen:


<p align="center">
<img src=./img/ensayo_tension.jpg width="20%">
</p>

Ahora, se quiere encontrar el esfuerzo de tracción, el cual está definido como:
$$\sigma = \frac{F}{A_0}$$

Siendo $F$ la fuerza que está siendo aplicada y $A$ el área de la sección transversal, de la cual su fórmula para este caso está dada por:

$$A_0 = ancho\times espesor \rightarrow A_0 = W\times T$$
<p align="center">
<img src=./img/tension.jpg width="20%">
</p>

Ahora, para la deformación unitaria, definida como el cambio entre la longitud inicial y la final, sobre la inicial:

$$\epsilon = \frac{l_i - l_0}{l_0} $$


<div class='alert alert-block alert-info'>   

<i class='fa fa-info-circle' aria-hidden='true'></i>
Se quiere reconstruir la curva de Esfuerzo - Deformación del HDPE, la probeta tiene un ancho de 13.92 mm, un espesor de 3.14 y una longitud inicial de 25 mm. 

Como puede observar, se tienen comportamientos distintos en cada una de las secciones, al igual que en el ejercicio anterior, debe poder recrear la curva utilizando las diferentes funciones de interpolación (si realizó las funciones de la sección pasada las puede utilizar en este punto). Los datos que se suministran para el ejercicio son de fuerza, el tiempo y la extensión de tracción medida por el equipo. 

**2.1. Importe el csv que se encuentra en el repositorio**
- Tenga en cuenta que se tienen dos headers, revise la documentación para presentar un dataFrame con ambos. 
- Agregue **encoding='utf-8'** para que se cargue correctamente el archivo. 

**2.2. Creación DataFrame <br>**
- Con los datos dados cree un DataFrame con los cuales sus columnas sean: (Fuerza, Deformación).
- Encuentre el esfuerzo y la deformación **unitaria** de los datos dados con las fórmulas dadas, estos datos deben quedar en el DataFrame creado como nuevas columnas. 
- **Importante:** Para la deformación unitaria ya se le dan los datos de cuánto se deformó por lo que debe tomar la longitud inicial y dividirla entre el número respectivo. 

**2.3. Grafique los puntos y comente cuáles funciones junto con su librería utilizaría para interpolar cada sección junto con una justificación <br>**
**2.4. Interpole los datos <br>**
**2.5. Grafique la curva con los datos dados y la de los datos del csv.**
- Con esto puede comprobar si su interpolación fue correcta y se acerca a la original.


**Recomendaciones:**
- Divida correctamente los datos para cada sección. Una ayuda para esto se encuentra en la presentación de ppt que se encuenta en bloque neón y en github.

</div>


In [10]:
# 1. Importe los datos del csv [Tenga en cuenta que se tienen 2 headers]


,Tiempo,Carga,Extensión de tracción
,s,N,mm
0,0.0,0.25441,0.00011
1,0.1,17.75244,0.01695
2,0.2,52.51681,0.05468
3,0.3,82.20963,0.09200
4,0.4,108.18626,0.12843


In [11]:
# 2. Datos iniciales


In [12]:
# Calcular área


43.708800000000004

## **Punto 3 - Sección Raíces**
----

<p align="center">
<img src=./img/Ejercicio_2.png width="30%">
</p>


En la figura anterior tiene dos curvas con las cuales debe:
1. Obtener datos de x y y de cada una
2. Interpole los datos (Esto con el fin de aproximar mejor la solución)
2. Utilizando la librería polyfit de numpy y los datos del literal anterior encuentre las ecuaciones de las curvas.
3. Teniendo las ecuaciones diga cuál sería la raíz para este caso. 
4. Encuentre la raíz utilizando las ecuaciones encontradas. 


In [32]:
# Puntos con las curvas


# Ineterpolación de los puntos



## Punto 4
----

Uno de los puntos importantes que se obtienen con la curva esfuerzo vs deformación es el esfuerzo de fluencia $S_{y}$, este se define como el esfuerzo máximo que se puede desarrollar en un material sin causar una deformación plástica[^1]. Este se puede encontrar trazando una línea paralela y que parte desde el 0.002 en el eje x que es el que tiene la deformación, cuando se interseca con la curva original del material es cuando se obtiene su valor. 

1. Utilizando algunos puntos del primer tramo del punto 2, encuentre el módulo de Young. 
- Para que no se tengan errores con la función de *polyfit* utilizar con los valores escogidos utilice np.array(x).reshape(-1,).
2. Se quiere encontrar el esfuerzo de Fluencia para lo cual de los puntos que seleccionó anteriormente al ser un array: 
- 2.1. En una nueva variable sumele a la deformación 0.002.
- 2.2. Encuentre la ecuación utilzando los nuevos x (deformacion) y el y (Esfuerzo) del literal 1.
- 2.3. Para realizar la paralela y que cruce la curva inicial defina una variable que van a ser los nuevos puntos x utilizando como punto inicial el x[0] de la deformación desplazada. 
- 2.4. Evalúe los valores del 2.3. en la ecuación del punto 2.2.

3. Grafique la recta encontrada junto con la gráfica de esfuerzo deformación del tramo correspondiente.
4. Como puede evidenciar en donde se corta con la curva original si se delimita entre 0.005 y 0.01 se tiene una recta. Encuentre la ecuación de esta recta.
5. En este caso se encuentra que se tiene un problema de raices, siendo esta en donde se intersecan las rectas. Encuentre el esfuerzo de fluencia del material y compare con la literatura. 


[^1]: [Instron] (https://www.instron.com/es-es/resources/glossary/y/yield-strength)